In [1]:
# conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db") 
# db = pd.read_sql_query("select * from clinical_trials ORDER BY [Primary Completion Date] ASC;", conn)
# conn.close()
import pandas as pd
from IPython.display import HTML
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)  
pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_colwidth', 150)

In [ ]:
# Cols = ['symbol','Sponsor/Collaborators','Completion Date','Primary Completion Date',
#         'Title','Interventions','Status','Study Results','Conditions',
#         'Interventions','Outcome Measures','Enrollment','Last Update Posted',
#         'Acronym','NCT Number','URL']
# df_CT = df_CT[Cols]

# df_companies.drop(['new'], axis=1)
# df_CT = pd.merge(df_CT, df_companies, on="symbol")
# df_CT = df_CT.sort_values(by = ['Completion Date'])
# df_CT = df_CT.reset_index(drop=True)

In [ ]:
df_CT.sort_values(by = ['Completion Date'])

In [ ]:
df_companies

In [ ]:
    conn = sqlite3.connect("BiotechDatabase.db")
    df_companies = pd.read_sql_query("select symbol,Company,[My Notes Date],[Revisit Date],[My Notes],[Score (1-10)],[Last Updated],[Mkt. Cap],Price from biotech;", conn)
    df_companies

In [ ]:
    conn = sqlite3.connect("BiotechDatabase.db")
    df_companies = pd.read_sql_query("select symbol,Company,[My Notes Date],[Revisit Date],[My Notes],[Score (1-10)],[Last Updated],[Mkt. Cap],Price from biotech;", conn)
    df_companies

In [ ]:
df_companies

In [ ]:
df_companies.drop(['new'], axis=1)

In [ ]:
df_CT

In [ ]:
"Rank","NCT Number","Title","Acronym","Status","Study Results","Conditions","Interventions","Outcome Measures","Sponsor/Collaborators","Gender","Age","Phases","Enrollment","Funded Bys","Study Type","Study Designs","Other IDs","Start Date","Primary Completion Date","Completion Date","First Posted","Results First Posted","Last Update Posted","Locations","Study Documents","URL"



import csv, sqlite3

con = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db") # change to 'sqlite:///your_filename.db'
cur = con.cursor()
cur.execute("CREATE TABLE t (col1, col2);") # use your column names here

with open('data.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['col1'], i['col2']) for i in dr]

cur.executemany("INSERT INTO t (col1, col2) VALUES (?, ?);", to_db)
con.commit()
con.close()

In [2]:
from bs4 import BeautifulSoup
import requests
import sys

# Access page
cik = '1369290'
type = '10-q'
dateb = '20200927'


# Obtain HTML for search page
base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}"
edgar_resp = requests.get(base_url.format(cik, type, dateb))
edgar_str = edgar_resp.text

# Find the document link
doc_link = ''
soup = BeautifulSoup(edgar_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile2')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 3:
        if '2020' in cells[3].text:
            doc_link = 'https://www.sec.gov' + cells[1].a['href']

# Exit if document link couldn't be found
if doc_link == '':
    print("Couldn't find the document link")
    sys.exit()

# Obtain HTML for document page
doc_resp = requests.get(doc_link)
doc_str = doc_resp.text

# Find the XBRL link
xbrl_link = ''
soup = BeautifulSoup(doc_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile', summary='Data Files')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 3:
        if 'INS' in cells[3].text:
            xbrl_link = 'https://www.sec.gov' + cells[2].a['href']

# Obtain XBRL text from document
xbrl_resp = requests.get(xbrl_link)
xbrl_str = xbrl_resp.text

# Find and print stockholder's equity
soup = BeautifulSoup(xbrl_str, 'lxml')
tag_list = soup.find_all()
for tag in tag_list:
    if tag.name == 'us-gaap:stockholdersequity':
        print("Stockholder's equity: " + tag.text) 

Stockholder's equity: 11782757
Stockholder's equity: 1824708
Stockholder's equity: 57
Stockholder's equity: 57957097
Stockholder's equity: -56125982
Stockholder's equity: -6464
Stockholder's equity: 281
Stockholder's equity: 71716915
Stockholder's equity: -59927975
Stockholder's equity: -6464
Stockholder's equity: 41
Stockholder's equity: 51721834
Stockholder's equity: -45289526
Stockholder's equity: -6464
Stockholder's equity: 6425885
Stockholder's equity: 56
Stockholder's equity: 57538633
Stockholder's equity: -48011033
Stockholder's equity: -6464
Stockholder's equity: 9521192


In [ ]:
table_tag = soup.find('table')



In [ ]:
soup

In [ ]:
https://www.sec.gov/Archives/edgar/data/1137883/000114420419030587/tv523404_424b5.htm

In [ ]:
for row in rows[1]:
    cells = row.find_all('td')
    if len(cells) > 3:
        if '2020' in cells[3].text:
            doc_link = 'https://www.sec.gov' + cells[1].a['href']
            return(doc_link)

In [3]:
import sqlite3


symbol = 'MYO'
symbol = symbol.upper()
symbols = symbol.split(',')

try:
    conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db") 
    cur = conn.cursor()
    sql_select_query = "select CIK from biotech where symbol = ?"
    cur.execute(sql_select_query,(symbol,))
    cik = str(cur.fetchone()[0])
    conn.commit()
    cur.close()

except sqlite3.Error as error:
    print("Failed to select symbol from biotech", error)
finally:
    if (conn):
        conn.close() 


if cik == 'None': 
    ticker = Ticker(symbols, asynchronous=True)
    df_secfilings = ticker.sec_filings
    if type(df_secfilings) is dict:
        print('No SEC Filings data available')
    else:
        df_secfilings = df_secfilings.drop(['maxAge', 'epochDate'], axis = 1)
        df_secfilings = df_secfilings.reset_index(level='row', drop = True)
        df_secfilings['edgarUrl'] = df_secfilings['edgarUrl'].apply(lambda x: '<a href="{0}" target="_blank">{0}</a>'.format(x))
        display(HTML(df_secfilings.to_html(escape=False)))

else:
    # Obtain HTML for search page
    base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&count=100" #&type={}&dateb={}"
    edgar_resp = requests.get(base_url.format(cik))
    edgar_str = edgar_resp.text

    # Find the document link
    doc_link = ''
    soup = BeautifulSoup(edgar_str, 'html.parser')
    table_tag = soup.find('table', class_='tableFile2')
    rows = table_tag.find_all('tr')

    #Table of all recent listings
    table = soup.find_all('table')
    filings_df = pd.read_html(str(table))[2]
    filings_df['URL']=''

    for index,row in zip(filings_df.index.to_list(),rows[1:]):

        cells = row.find_all('td')
        if len(cells) > 3:
    # #         if '2020' in cells[3].text:

            try:
                doc_link = 'https://www.sec.gov' + cells[1].a['href']
                filings_df.loc[index,'URL']=doc_link
            except:
                continue
    filings_df = filings_df.groupby('Filings').head(2).reset_index(drop=True)
    filings_df['URL'] = filings_df['URL'].apply(lambda x: '<a href="{0}" target="_blank">{0}</a>'.format(x))
    display(HTML(filings_df.to_html(escape=False)))

In [4]:
# Obtain HTML for search page
        base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&count=100" #&type={}&dateb={}"
        edgar_resp = requests.get(base_url.format(cik))
        edgar_str = edgar_resp.text

        # Find the document link
        doc_link = ''
        soup = BeautifulSoup(edgar_str, 'html.parser')
        table_tag = soup.find('table', class_='tableFile2')
        rows = table_tag.find_all('tr')

        #Table of all recent listings
        table = soup.find_all('table')
        filings_df = pd.read_html(str(table))[2]
        filings_df['URL']=''

        for index,row in zip(filings_df.index.to_list(),rows[1:]):

            cells = row.find_all('td')
            if len(cells) > 3:
        # #         if '2020' in cells[3].text:

                try:
                    doc_link = 'https://www.sec.gov' + cells[1].a['href']
                    filings_df.loc[index,'URL']=doc_link
                except:
                    continue
        filings_df = filings_df.groupby('Filings').head(2).reset_index(drop=True)
        filings_df['URL'] = filings_df['URL'].apply(lambda x: '<a href="{0}" target="_blank">{0}</a>'.format(x))
        display(HTML(filings_df.to_html(escape=False)))



    

IndentationError: unexpected indent (<ipython-input-4-5d6c5562bd65>, line 2)

In [5]:
#!/usr/bin/env python

# nix-shell -p python35Packages.pandas --command "python -i"

import xml.etree.ElementTree as ET
import pandas
import csv

def xml_to_dataframe(elements, attr_dict, prefix = ""):
    """Turn a list of elements into a Pandas DataFrame according to the
    attributes and conversions specified in 'attr_dict', which is a
    dictionary whose keys are the intended column name in that
    DataFrame and whose values are tuples containing (conversion
    function, XML path).  Conversion function is applied to the text
    at that XML path, and the XML path will have {0} replaced with
    optional 'prefix'.
    """
    def get_row(el):
        row = {}
        for k in attribs:
            fn, attr = attribs[k]
            row[k] = fn(el.find(attr.format(prefix)).text)
        return row
    # A row of dictionaries can be handled directly by Pandas:
    rows = [get_row(e) for e in elements]
    return pandas.DataFrame(rows)

# The dictionary of attributes from the XML file that we're interested
# in, and what types they should be.  (Specifically: key = desired
# column name, value = (conversion function, attribute).)
attribs = {
    "nameOfIssuer":           (str, "{0}nameOfIssuer"),
    "titleOfClass":           (str, "{0}titleOfClass"),
    "cusip":                  (str, "{0}cusip"),
    "value":                  (int, "{0}value"),
    "sshPrnamt":              (int, "{0}shrsOrPrnAmt/{0}sshPrnamt"),
    "sshPrnamtType":          (str, "{0}shrsOrPrnAmt/{0}sshPrnamtType"),
    "votingAuthoritySole":    (int, "{0}votingAuthority/{0}Sole"),
    "votingAuthorityShared":  (int, "{0}votingAuthority/{0}Shared"),
    "votingAuthorityNone":    (int, "{0}votingAuthority/{0}None"),
    "investmentDiscretion":   (str, "{0}investmentDiscretion"),
}

# Parse from an example XML file:
tree = ET.parse("baupost-20170811-form13fInfoTable.xml")
root = tree.getroot()

url_prefix = "{http://www.sec.gov/edgar/document/thirteenf/informationtable}"
df = xml_to_dataframe(
    root.findall("{0}infoTable".format(url_prefix)),
    attribs,
    url_prefix)

# Get table of (some) CUSIP IDs to ticker symbols.
cusip_to_symbol = pandas.read_csv("./cusip_to_symbol.csv")
# Note left join; we don't want to lose entries just because no symbol
# is available.
df = df.merge(cusip_to_symbol, how="left", on="cusip").fillna("")

# Dump to CSV and to screen:
df.to_csv("baupost_20170811_13f.csv", index=False)
pandas.set_option("display.width", None)
print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'baupost-20170811-form13fInfoTable.xml'

In [6]:
def parse(file, CIK):
    xml = open('C:\\13F\\sec_edgar_filings\\' + CIK + '\\13F-HR\\' + file).read()
    soup = BeautifulSoup(xml, 'lxml')
    try:
        date = datetime.datetime.strptime(soup.find('signaturedate').contents[0], '%m-%d-%Y')
        name = soup.find('name').contents[0]
    except:
        return
    cols = ['nameOfIssuer', 'cusip', 'value', 'sshPrnamt', 'sshPrnamtType', 'putCall']
    data = []
    print("Processing " + name + " (" + CIK + ") for date " + str(date))
    for infotable in soup.find_all(['ns1:infotable', 'infotable']):
        row = []
        for col in cols:
            data = infotable.find([col.lower(), 'ns1:' + col.lower()])
            row.append(data.text.strip() if d else 'NaN')
        row.append(date)
        row.append(CIK)
        row.append(name)
        data.append(row)
    df = pd.DataFrame(data)
    cols.append('date')
    cols.append('fund_cik')
    cols.append('fund')
    try:
        df.columns = cols
        return df
    except:
        return

In [7]:
xml = 'https://www.sec.gov/Archives/edgar/data/1315339/000131533920000004/form13fInfoTable.xml'
    
    
def parse(file, CIK):
    xml = 'https://www.sec.gov/Archives/edgar/data/1315339/000131533920000004/form13fInfoTable.xml'

    soup = BeautifulSoup(xml, 'lxml')
    try:
        date = datetime.datetime.strptime(soup.find('signaturedate').contents[0], '%m-%d-%Y')
        name = soup.find('name').contents[0]
    except:
        return
    cols = ['nameOfIssuer', 'cusip', 'value', 'sshPrnamt', 'sshPrnamtType', 'putCall']
    data = []
    print("Processing " + name + " (" + CIK + ") for date " + str(date))
    for infotable in soup.find_all(['ns1:infotable', 'infotable']):
        row = []
        for col in cols:
            data = infotable.find([col.lower(), 'ns1:' + col.lower()])
            row.append(data.text.strip() if d else 'NaN')
        row.append(date)
        row.append(CIK)
        row.append(name)
        data.append(row)
    df = pd.DataFrame(data)
    cols.append('date')
    cols.append('fund_cik')
    cols.append('fund')
    try:
        df.columns = cols
        return df
    except:
        return

In [8]:
import json

url = "https://api.filingspro.com/sec?ticker=MYO"
resp = requests.get(url)
text = resp.text

d = json.loads(text)
# rows list initialization 
rows = [] 

# appending rows 
for data in d["shelfs"]: 
    data_row = data['filings'] 
    time = data['effective'] 
      
    for row in data_row: 
        row['effective']= time 
        rows.append(row) 
  
# using data frame 
df = pd.DataFrame(rows)        
df = df.loc[:, ~(df.astype(str) == 'False').all()]
df = df.loc[:, ~(df.astype(str) == '').all()]
df


filingType  \
0      S-1MEF   
1       424B4   
2       S-1/A   
3       S-1/A   
4       S-1/A   
5       S-1/A   
6       S-1/A   
7         S-1   
8       424B5   
9       424B5   
10      424B5   
11        S-3   
12     S-1MEF   

                                                                                   documentUrl  \
0         https://www.sec.gov/Archives/edgar/data/1369290/000119312520030172/d850023ds1mef.htm   
1         https://www.sec.gov/Archives/edgar/data/1369290/000119312520033415/d850023d424b4.htm   
2           https://www.sec.gov/Archives/edgar/data/1369290/000119312520029794/d850023ds1a.htm   
3           https://www.sec.gov/Archives/edgar/data/1369290/000119312520029196/d850023ds1a.htm   
4           https://www.sec.gov/Archives/edgar/data/1369290/000119312520028287/d850023ds1a.htm   
5           https://www.sec.gov/Archives/edgar/data/1369290/000119312520022213/d850023ds1a.htm   
6           https://www.sec.gov/Archives/edgar/data/1369290/000119312520007317/d850023ds1a.htm   
7            https://www.sec.gov/Archives/edgar/data/1369290/000119312519315334/d850023ds1.htm   
8         https://www.sec.gov/Archives/edgar/data/1369290/000119312519032461/d699862d424b5.htm   
9         https://www.sec.gov/Archives/edgar/data/1369290/000119312519030874/d699862d424b5.htm   
10        https://www.sec.gov/Archives/edgar/data/1369290/000119312519030869/d685009d424b5.htm   
11           https://www.sec.gov/Archives/edgar/data/1369290/000119312518211426/d808637ds3.htm   
12  https://www.sec.gov/Archives/edgar/data/1369290/000121390017012768/fs1mef2017_myomoinc.htm   

        fileID  \
0   333-236360   
1   333-235538   
2   333-235538   
3   333-235538   
4   333-235538   
5   333-235538   
6   333-235538   
7   333-235538   
8   333-226045   
9   333-226045   
10  333-226045   
11  333-226045   
12  333-221817   

                                                                                                 fileUrl  \
0   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-236360&owner=exclude&count=40   
1   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-235538&owner=exclude&count=40   
2   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-235538&owner=exclude&count=40   
3   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-235538&owner=exclude&count=40   
4   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-235538&owner=exclude&count=40   
5   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-235538&owner=exclude&count=40   
6   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-235538&owner=exclude&count=40   
7   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-235538&owner=exclude&count=40   
8   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-226045&owner=exclude&count=40   
9   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-226045&owner=exclude&count=40   
10  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-226045&owner=exclude&count=40   
11  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-226045&owner=exclude&count=40   
12  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-221817&owner=exclude&count=40   

          date        mvphs        float  aggregateAmount  sharesOffered  \
0   2020-02-10                                    4775201              0   
1   2020-02-12                                   15000952        1660000   
2   2020-02-10                                   47808661         551000   
3   2020-02-10                                          0              0   
4   2020-02-07                                   15524425         551000   
5   2020-02-03                                   15525000        2100000   
6   2020-01-14                                   13500000              0   
7   2019-12-16   

In [9]:

df = pd.DataFrame(d["shelfs"])
df

effective                 expirationDate  expired  hasATM  \
0           2020-02-10                                   False   False   
1  2020-02-10 17:30:00  2023-02-09 17:30:00 +0000 UTC    False   False   
2  2018-07-16 16:00:00  2021-07-15 16:00:00 +0000 UTC    False    True   
3           2017-11-29                                   False   False   

   hasHiddenATM  isIPO  \
0         False  False   
1         False  False   
2          True  False   
3         False  False   

                                                                                                                                                 filings  \
0  [{'filingType': 'S-1MEF', 'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1369290/000119312520030172/d850023ds1mef.htm', 'fileID': '333-23...   
1  [{'filingType': '424B4', 'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1369290/000119312520033415/d850023d424b4.htm', 'fileID': '333-235...   
2  [{'filingType': '424B5', 'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1369290/000119312519032461/d699862d424b5.htm', 'fileID': '333-226...   
3  [{'filingType': 'S-1MEF', 'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1369290/000121390017012768/fs1mef2017_myomoinc.htm', 'fileID': '...   

       fileID baseType latestFilingDate  show  foreign  
0  333-236360   S-1MEF       2020-02-10  True    False  
1  333-235538      S-1       2020-02-12  True    False  
2  333-226045      S-3       2019-02-08  True    False  
3  333-221817   S-1MEF       2017-11-29  True    False

In [ ]:
pd.concat((pd.read_json(data) for data in d["shelfs"]), axis=0)

In [ ]:
for data in d:
    print(data)

In [ ]:
df

In [ ]:
dilution('MYO')

In [10]:
import contextlib
import numpy as np
import pandas as pd
import pandas.io.formats.format as pf


@contextlib.contextmanager
def custom_formatting():
    orig_float_format = pd.options.display.float_format
    orig_int_format = pf.IntArrayFormatter

    pd.options.display.float_format = '{:0,.2f}'.format
    class IntArrayFormatter(pf.GenericArrayFormatter):
        def _format_strings(self):
            formatter = self.formatter or '{:,d}'.format
            fmt_values = [formatter(x) for x in self.values]
            return fmt_values
    pf.IntArrayFormatter = IntArrayFormatter
    yield
    pd.options.display.float_format = orig_float_format
    pf.IntArrayFormatter = orig_int_format

In [11]:
%run definitions.py

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
#https://www.sec.gov/Archives/edgar/data/1369290/000156459020038994/R2.htm
    
base_url = "https://www.sec.gov/Archives/edgar/data/1369290/000156459020038994/R2.htm"
edgar_resp = requests.get(base_url)
edgar_str = edgar_resp.text

# Find the document link
doc_link = ''
soup = BeautifulSoup(edgar_str, 'html.parser')
# table_tag = soup.find('table', class_='tableFile2')
# rows = table_tag.find_all('tr')

#Table of all recent listings
table = soup.find_all('table')
bs_df = pd.read_html(str(table))[0]

In [13]:
bs_df 

Condensed Consolidated Balance Sheets - USD ($)  \
0                                                                                                                                         Current Assets:   
1                                                                                                                               Cash and cash equivalents   
2                                                                                                                                Accounts receivable, net   
3                                                                                                                                        Inventories, net   
4                                                                                                                              Prepaid expenses and other   
5                                                                                                                                    Total Current Assets   
6                                                                                                                                         Restricted cash   
7                                                                                                                                 Deferred offering costs   
8                                                                                                                                          Equipment, net   
9                                                                                                                                            Total Assets   
10                                                                                                                                   Current Liabilities:   
11                                 Convertible note payable, net of discount of $71,360 and $676,703 at June 30, 2020 and December 31, 2019, respectively   
12                                                                                                                  Accounts payable and accrued expenses   
13                                                                                                                                 Derivative liabilities   
14                                                                                                                                       Deferred revenue   
15                                                                                                                              Total Current Liabilities   
16                                                                                        Long-term debt, net of discount of $36,169 at December 31, 2019   
17                                                                                                                       Other long-term debt liabilities   
18                                                                                                                                       Deferred revenue   
19                                                                                                                                      Total Liabilities   
20                                                                                                                          Commitments and Contingencies   
21                                                                                                                                  Stockholders’ Equity:   
22  Common stock par value $0.0001 per share, 100,000,000 shares authorized; 3,223,876 and 574,524 shares issued as of June 30, 2020 and December 31, ...   
23                                                                                                                             Additional paid-in capital   
24                                                                                                                                    Accumulated deficit   
25                      

In [14]:
#Tutorial here: https://www.scrapingbee.com/blog/selenium-python/
import datetime
today = datetime.datetime.now()
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import random
from random import randint
from time import sleep

sleep(randint(1,60))

DRIVER_PATH = '/Users/ralph/Biotech/chromedriver'
#This will launch Chrome in headfull mode (like a regular Chrome, which is controlled by your Python code). 
#You should see a message stating that the browser is controlled by an automated software.
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

#In order to run Chrome in headless mode (without any graphical user interface), 
#to run it on a server for example, add the code below and replace driver above with the one below:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
options.headless = True
# options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

#Tutorial here:
driver.get("https://finviz.com/insidertrading.ashx") #for all insider activities
#driver.get("https://finviz.com/insidertrading.ashx?tc=1") #for buyers
#driver.get("https://finviz.com/insidertrading.ashx?tc=2") #for sellers
#driver.get("https://finviz.com/insidertrading.ashx?tc=4") #for options
time.sleep(3)

soup = BeautifulSoup(driver.page_source, 'lxml')
#Always quit at the end of scraping
driver.quit()

table = soup.find_all('table')

finviz_df = pd.read_html(str(table))[7] 
headers = ['symbol','Owner','Relationship','Date','Transaction','Cost','Shares','Value ($)','Shares Total','SEC Form 4']
finviz_df  = pd.DataFrame(finviz_df.values[1:], columns=headers)
# finviz_df = finviz_df.set_index('symbol')
finviz_df['SEC Form 4'] = str(today.year) +' '+ finviz_df['SEC Form 4'].astype(str)
finviz_df['SEC Form 4'] = finviz_df['SEC Form 4'].astype('datetime64[ns]')
finviz_df['Date'] = str(today.year) +' '+ finviz_df['Date'].astype(str)
finviz_df['Date'] = finviz_df['Date'].astype('datetime64[ns]')
finviz_df['Cost'] = finviz_df['Cost'].apply(pd.to_numeric, errors='ignore', downcast='float')
finviz_df['Shares'] = finviz_df['Shares'].fillna(0).astype(int)
finviz_df['Shares Total'] = finviz_df['Shares Total'].fillna(0).astype(int)
finviz_df['Value ($)'] = finviz_df['Value ($)'].fillna(0).astype(int)
finviz_df['Cost'] = finviz_df['Cost'].apply(lambda x: round(x, 2))
finviz_df = finviz_df.sort_values(['SEC Form 4'], ascending = False)
finviz_df = finviz_df.reset_index(drop=True)
finviz_df

#Making a list of the new insiders symbols
symbols_finviz=finviz_df.symbol.tolist()

#Getting the biotech database and the symbol list
try:
    conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db")
    symbols_df = pd.read_sql_query("select symbol from biotech;", conn) 
    symbols_db = symbols_df['symbol'].tolist()
except sqlite3.Error as error:
    print("Failed to access the BiotechDatabase file", error)  
finally:
    if (conn):
        conn.close()

        
#Isolating the biotech insiders symbols dataframe showing on finviz.com
biotech_finviz = sorted(list(set(symbols_db).intersection(symbols_finviz)))
biotech_finviz_new_df= finviz_df[finviz_df['symbol'].isin(biotech_finviz)].copy()
# biotech_finviz_new_df['SEC Form 4'] = str(today.year) +' '+ biotech_finviz_new_df['SEC Form 4'].astype(str)
# biotech_finviz_new_df['SEC Form 4'] = biotech_finviz_new_df['SEC Form 4'].astype('datetime64[ns]')
# biotech_finviz_new_df['Date'] = str(today.year) +' '+ biotech_finviz_new_df['Date'].astype(str)
# biotech_finviz_new_df['Date'] = biotech_finviz_new_df['Date'].astype('datetime64[ns]')
# biotech_finviz_new_df['Cost'] = biotech_finviz_new_df['Cost'].apply(pd.to_numeric, errors='ignore', downcast='float')
# biotech_finviz_new_df['Shares'] = biotech_finviz_new_df['Shares'].fillna(0).astype(int)
# biotech_finviz_new_df['Shares Total'] = biotech_finviz_new_df['Shares Total'].fillna(0).astype(int)
# biotech_finviz_new_df['Value ($)'] = biotech_finviz_new_df['Value ($)'].fillna(0).astype(int)
# biotech_finviz_new_df['Cost'] = biotech_finviz_new_df['Cost'].apply(lambda x: round(x, 2))
biotech_finviz_new_df = biotech_finviz_new_df.sort_values(['SEC Form 4'], ascending = False)
biotech_finviz_new_df = biotech_finviz_new_df.reset_index(drop=True)

try:
    conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db")
    biotech_finviz_new_df.to_sql('temp_insiders_bio', conn, if_exists='replace', index=False)
    finviz_df.to_sql('temp_insiders_all', conn, if_exists='replace', index=False)

    cur = conn.cursor()
    cur.executescript('''
                    INSERT OR IGNORE INTO insiders_all
                    SELECT * FROM temp_insiders_all;
                    SELECT * FROM insiders_all
                    ORDER BY [SEC Form 4] DESC;
                    INSERT OR IGNORE INTO insiders_bio
                    SELECT * FROM temp_insiders_bio;
                    SELECT * FROM insiders_bio
                    ORDER BY [SEC Form 4] DESC;
                    ''')
    conn.commit()
    cur.close()
except sqlite3.Error as error:
    print("Failed to connect to the insiders tables", error)  
finally:
    if (conn):
        conn.close()

/Users/ralph/Biotech/venv/lib/python3.8/site-packages/pandas/core/generic.py:2602: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [15]:
import datetime
today = datetime.datetime.now()
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import random
from random import randint
from time import sleep
import requests 

# sleep(randint(1,60))
url = "https://fintel.io/activists"
resp = requests.get(url)
fintel_text = resp.text

# Find the document link
doc_link = ''
soup = BeautifulSoup(fintel_text, 'html.parser')
# table_tag = soup.find('table', class_='tableFile2')
# rows = table_tag.find_all('tr')

#Table of all recent listings
table = soup.find('table')
fintel_df = pd.read_html(str(table))[0]
fintel_df

fintel_df[['Symbol','Company']] = fintel_df['Target'].str.split('/ ',expand=True)

In [16]:
fintel_df

Date   Form  \
0   2021-01-22  13G/A   
1   2021-01-22  13D/A   
2   2021-01-22  13G/A   
3   2021-01-22  13G/A   
4   2021-01-22    13G   
5   2021-01-22  13D/A   
6   2021-01-22    13G   
7   2021-01-22    13G   
8   2021-01-22  13G/A   
9   2021-01-22  13G/A   
10  2021-01-22  13D/A   
11  2021-01-22  13G/A   
12  2021-01-22  13D/A   
13  2021-01-22  13G/A   
14  2021-01-22  13G/A   
15  2021-01-22  13D/A   
16  2021-01-22  13G/A   
17  2021-01-22  13D/A   
18  2021-01-22  13G/A   
19  2021-01-22    13G   
20  2021-01-22  13G/A   
21  2021-01-22    13D   
22  2021-01-22    13G   
23  2021-01-22    13G   
24  2021-01-22    13G   
25  2021-01-22  13D/A   
26  2021-01-22  13D/A   
27  2021-01-22    13G   
28  2021-01-22    13D   
29  2021-01-22  13D/A   
30  2021-01-22  13D/A   
31  2021-01-22    13G   
32  2021-01-22    13G   
33  2021-01-22    13G   
34  2021-01-22    13G   
35  2021-01-22  13G/A   
36  2021-01-22  13G/A   
37  2021-01-22  13G/A   
38  2021-01-22  13D/A   
39  2021-01-22    13G   
40  2021-01-22  13G/A   
41  2021-01-22    13G   
42  2021-01-22    13G   
43  2021-01-22  13D/A   
44  2021-01-22    13G   
45  2021-01-22    13G   
46  2021-01-22    13G   
47  2021-01-22  13G/A   
48  2021-01-22  13G/A   
49  2021-01-22  13G/A   

                                                                                                          Target  \
0                                                                                       ARTH / Arch Therapeutics   
1                                                                                           AHNR / Athena Silver   
2                                                                                              PLXP / PLx Pharma   
3                                                                                   VTGN / VistaGen Therapeutics   
4                                                                          BTI / British American Tobacco p.l.c.   
5                                                                                           CMD / Cantel Medical   
6                                                                              BTRS / BTRS Holdings Inc. Class 1   
7                                                                            ADERU / 26 Capital Acquisition Unit   
8                                                                           SEII / Sharing Economy International   
9                                                                                         INMB / INmune Bio Inc.   
10                                                                                            QGEN / QIAGEN N.V.   
11                                                                                              NTST / NetSTREIT   
12                                                                                  NTLA / Intellia Therapeutics   
13                                                                                             AMZN / Amazon.com   
14                                                                                        JACK / Jack in the Box   
15                                                                                             PCYO / Pure Cycle   
16                                                                                  LQDA / Liquidia Technologies   
17                                                                                                 PCTI / PC-Tel   
18                                                                                                IVC / Invacare   
19                                                                           PAQCU / Provident Acquisition Units   
20                                                                                     IMPX / AEA-Bridges Impact   
21                                                                             BTRS / BTRS Holdings Inc. Class 1   
22                                                                

In [19]:
fintel_df = pd.read_html(str(table))[0]
fintel_df

df[['V','allele']] = df['V'].str.split('-',expand=True)

Date   Form                                             Target  \
0   2020‑11‑06  13D/A                  TPTX / Turning Point Therapeutics   
1   2020‑11‑06    13D                         CRC / California Resources   
2   2020‑11‑06    13D                     / GLOBAL INDEMNITY GROUP LLC-A   
3   2020‑11‑06  13D/A                                  MOBL / MobileIron   
4   2020‑11‑06    13D                                     GLTO / Galecto   
5   2020‑11‑06  13D/A                         TESS / Tessco Technologies   
6   2020‑11‑06    13G  NBA.U / New Beginnings Acquisition Units, each...   
7   2020‑11‑06    13D                          ROOT / Root, Inc. Class A   
8   2020‑11‑06    13D                         APVO / Aptevo Therapeutics   
9   2020‑11‑06  13D/A                                     CVNA / Carvana   
10  2020‑11‑06    13D                         CRC / California Resources   
11  2020‑11‑06    13G                                 EPIX / ESSA Pharma   
12  2020‑11‑06  13D/A      TMTS / TERWIN MORTGAGE TRUST TMTS 2003 6HE A3   
13  2020‑11‑06    13D                           SLDB / Solid Biosciences   
14  2020‑11‑06    13D                           MCF / Contango Oil & Gas   
15  2020‑11‑06    13G                         CRC / California Resources   
16  2020‑11‑06    13D   ARYB / ARYA Sciences Acquisition Corp II Class A   
17  2020‑11‑06    13D            / Conyers Park II Acquisition , Class A   
18  2020‑11‑06    13G                                       GPRO / GoPro   
19  2020‑11‑06    13G                      JKS / JinkoSolar Holdings Co.   
20  2020‑11‑06    13G            NOA / North American Construction Group   
21  2020‑11‑06  13D/A                        RPLA.U / Replay Acquisition   
22  2020‑11‑06  13D/A                        RPLA.U / Replay Acquisition   
23  2020‑11‑06  13G/A                           INVT / Inventergy Global   
24  2020‑11‑06  13G/A                  ADSW / Advanced Disposal Services   
25  2020‑11‑06  13D/A                                   ULBI / Ultralife   
26  2020‑11‑06  13D/A                      NANX / Nanophase Technologies   
27  2020‑11‑06  13G/A                     ESXB / Community Bankers Trust   
28  2020‑11‑06  13G/A                  UAL / United Continental Holdings   
29  2020‑11‑06  13G/A                                    WHR / Whirlpool   
30  2020‑11‑06  13G/A                               HMNF / HMN Financial   
31  2020‑11‑06    13G                BKCC / BlackRock Capital Investment   
32  2020‑11‑06  13G/A                            WBBW / WESTBURY BANCORP   
33  2020‑11‑06  13G/A             CRBP / Corbus Pharmaceuticals Holdings   
34  2020‑11‑06    13G                           MCF / Contango Oil & Gas   
35  2020‑11‑06    13G       TPZ / Tortoise Power and Infrastructure Fund   
36  2020‑11‑06  13D/A                            BITA / Bitauto Holdings   
37  2020‑11‑06    13G                    BHSEU / Bull Horn Holdings Unit   
38  2020‑11‑06  13D/A                               LEJU / Leju Holdings   
39  2020‑11‑06    13G                       OIIM / O2Micro International   
40  2020‑11‑05    13D                 HSDT / Helius Medical Technologies   
41  2020‑11‑05  13D/A                 IMXI / International Money Express   
42  2020‑11‑05  13D/A                                   CNTG / Centogene   
43  2020‑11‑05  13G/A                     KNSA / Kiniksa Pharmaceuticals   
44  2020‑11‑05  13D/A                                        XRX / Xerox   
45  2020‑11‑05    13D                                     GLTO / Galecto   
46  2020‑11‑05  13D/A                                    HLF / Herbalife   
47  2020‑11‑05  13D/A                                   HLIX / Helix TCS   
48  2020‑11‑05    13D                                 TCP / TC Pipelines   
49  2020‑11‑05    13D                         SENS / Senseonics Holdings   

                                   Lead Investor  Unnamed: 4  MarketCap ($M)  \
0                            GLAXOSMITHKLINE PLC     

In [17]:
table = soup.find('table')

In [18]:
hf_df

NameError: name 'hf_df' is not defined

In [7]:


# df_insiders_all = pd.DataFrame()


# url = 'https://www.marketwatch.com/investing/stock/'+str(a)+'/insideractions'
#         #print(url1)
#         res1 = requests.get(url1)
#         #print(res1)
#         soup1 = BeautifulSoup(res1.text, 'html.parser')
#         summary = soup1.find("div", {"id": "insiderTransactionSummary"})
#         try:
#             pd.read_html(str(summary), header =0, index_col=0)
#         except ValueError:
#             print('-'*60)
#             print('-'*60)
#             print('')
#             print ('INSIDER TRANSACTIONS:')
#             print("No Insider Transaction data available for "+symbol)
#             pass

In [20]:
import feedparser
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import sqlite3
import re

feed = feedparser.parse('https://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent&CIK=&type=SC%2013&company=&dateb=&owner=include&start=0&count=100&output=atom')


In [21]:
posts = [] # list of posts [(title1, link1, summary1), (title2, link2, summary2) ... ]
for post in feed.entries:
    posts.append((post.title, post.link, post.summary))
    

df = pd.DataFrame(posts, columns=['title', 'link', 'summary']) # pass data to init
df = df[~df.title.str.contains("Filed by")]
df['Form']= df["title"].str.split(" - ", n = -1, expand = True)[0]
df["Target"] = df["title"].apply(lambda st: st[st.find("- ")+1:st.find("(")])
df['Date']= df["summary"].apply(lambda st: st[st.find("> ")+2:st.find(" <b")])
df["CIK"] = df["title"].str.extract('\((\d+)\)')
df["CIK"] = df["CIK"].fillna(0).astype(int)

df = df.drop(columns=['summary'])
df['Lead_Investor']=''

cols = list(df)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Date')))
cols.insert(1, cols.pop(cols.index('Form')))
cols.insert(2, cols.pop(cols.index('Target')))
cols.insert(3, cols.pop(cols.index('Lead_Investor')))
df = df[cols]
df = df.reset_index(drop=True)


for link in df['link']:
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')
    soup2 = soup.find_all("span", class_="companyName")
    soup2 = soup.find_all("span", class_="companyName")
    soup3 = str(soup2)
    df.loc[df['link'] == link,'Lead_Investor'] = re.findall(r'Name\"\>(.+?) \(F',soup3)
    try:
        soup_hrefs= soup.find_all('a', href=True)
        soup_href = str(soup_hrefs)
        form_link= 'https://www.sec.gov/Archives/'+re.findall(r'Archives(.+?)htm',soup_href)[0]+'htm'
        df.loc[df['link'] == link,'Form_Link'] = form_link
    except:
        pass

# for link in df['link']:
#     res = requests.get(link)
#     soup = BeautifulSoup(res.text, 'html.parser')
#     soup2 = soup.find("span", class_="companyName").text
    
#     df.loc[df['link'] == link,'CIK2'] = re.findall(r'CIK\: (.+?) \(',soup2)[0]
#     df.loc[df['link'] == link,'Lead_Investor'] = soup2.split(" (")[0]

df


Date       Form                                   Target  \
0   2021-01-22   SC 13D/A                      ATHENA SILVER CORP    
1   2021-01-22   SC 13G/A                 Arch Therapeutics, Inc.    
2   2021-01-22   SC 13G/A             VistaGen Therapeutics, Inc.    
3   2021-01-22     SC 13G         British American Tobacco p.l.c.    
4   2021-01-22   SC 13G/A                         PLx Pharma Inc.    
5   2021-01-22   SC 13D/A                     CANTEL MEDICAL CORP    
6   2021-01-22     SC 13G                      BTRS Holdings Inc.    
7   2021-01-22     SC 13G            FinTech Acquisition Corp. IV    
8   2021-01-22     SC 13G            26 Capital Acquisition Corp.    
9   2021-01-22  SC 13E3/A            AeroGrow International, Inc.    
10  2021-01-22   SC 13G/A      SHARING ECONOMY INTERNATIONAL INC.    
11  2021-01-22   SC 13G/A                        Inmune Bio, Inc.    
12  2021-01-22   SC 13D/A                             QIAGEN N.V.    
13  2021-01-22   SC 13D/A             Intellia Therapeutics, Inc.    
14  2021-01-22   SC 13G/A                         NETSTREIT Corp.    
15  2021-01-22   SC 13G/A      Montes Archimedes Acquisition Corp    
16  2021-01-22   SC 13G/A                          AMAZON COM INC    
17  2021-01-22   SC 13G/A               JACK IN THE BOX INC /NEW/    
18  2021-01-22   SC 13D/A                              PC TEL INC    
19  2021-01-22   SC 13G/A                           Liquidia Corp    
20  2021-01-22   SC 13D/A                         PURE CYCLE CORP    
21  2021-01-22   SC 13G/A                           INVACARE CORP    
22  2021-01-22     SC 13G             Provident Acquisition Corp.    
23  2021-01-22     SC 13G         Golden Falcon Acquisition Corp.    
24  2021-01-22   SC 13G/A                AEA-Bridges Impact Corp.    
25  2021-01-22     SC 13G                Dine Brands Global, Inc.    
26  2021-01-22     SC 13D                      BTRS Holdings Inc.    
27  2021-01-22     SC 13G                     TIVITY HEALTH, INC.    
28  2021-01-22     SC 13G                            AC Immune SA    
29  2021-01-22   SC 13D/A               PLATINUM GROUP METALS LTD    
30  2021-01-22     SC 13G                       LEVI STRAUSS & CO    
31  2021-01-22   SC 13D/A                   CONTANGO OIL & GAS CO    
32  2021-01-22     SC 13D          Newtek Business Services Corp.    
33  2021-01-22   SC 13D/A                           OncoCyte Corp    
34  2021-01-22   SC 13D/A             Mid-Con Energy Partners, LP    
35  2021-01-22     SC 13G   Northern Genesis Acquisition Corp. II    
36  2021-01-22     SC 13G                      Switchback II Corp    
37  2021-01-22     SC 13G                Avanti Acquisition Corp.    
38  2021-01-22     SC 13G                     KL Acquisition Corp    
39  2021-01-22   SC 13G/A                          1PM Industries    
40  2021-01-22   SC 13G/A                         Deep Down, Inc.    
41  2021-01-22   SC 13G/A                  Conifer Holdings, Inc.    
42  2021-01-22   SC 13D/A          DESTINY MEDIA TECHNOLOGIES INC    
43  2021-01-22     SC 13G                        Port Capital LLC    
44  2021-01-22   SC 13G/A              CardioGenics Holdings Inc.    
45  2021-01-22     SC 13G            Generations Bancorp NY, Inc.    
46  2021-01-22     SC 13G                          Sunstock, Inc.    
47  2021-01-22   SC 13D/A                      MAGELLAN GOLD Corp    
48  2021-01-22   SC 13G/A                 Stone Point Credit Corp    
49  2021-01-22     SC 13G               TITAN PHARMACEUTICALS INC    
50  2021-01-22     SC 13G                                 Dogness    
51  2021-01-22     SC 13G                IDEAYA Biosciences, Inc.    

                                        Lead_Investor  \
0                                        Gibbs John D   
1                               Parker Michael Andrew   
2                             Rosalind Advisors, Inc.   
3                             Portfolio Services Ltd.   
4             

In [ ]:
len(feed.entries)
feed

In [313]:
for link in df['Form_Link'].head(1):
    print(link)
    try:
        res = requests.get(link)
        file_soup = BeautifulSoup(res.text, 'html.parser')
#     soup2 = soup.find_all("span", class_="companyName")
#     soup2 = soup.find_all("span", class_="companyName")
#     soup3 = str(soup2)
#     df.loc[df['link'] == link,'Lead_Investor'] = re.findall(r'Name\"\>(.+?) \(F',soup3)
#     try:
#         soup_hrefs= soup.find_all('a', href=True)
#         soup_href = str(soup_hrefs)
#         form_link= 'https://www.sec.gov/Archives/'+re.findall(r'Archives(.+?)htm',soup_href)[0]+'htm'
#         df.loc[df['link'] == link,'Form_Link'] = form_link
    except:
        pass

https://www.sec.gov/Archives//edgar/data/1091223/000119312520302746/d39189dsc13da.htm


In [314]:
file_soup
# file_soup_p=file_soup.find_all('p')
# file_soup = str(file_soup)
# re.findall(r'AGGREGATE(.+?)td',file_soup)
# table = file_soup.find('table')
# table_df = pd.read_html(str(table))[0]
# table_df
# soup_hrefs= soup.find_all('a', href=True)
# soup_href = str(soup_hrefs)
# form_link= 'https://www.sec.gov/Archives/'+re.findall(r'Archives(.+?)htm',soup_href)[0]+'htm'
# df.loc[df['link'] == link,'Form_Link'] = form_link

/html/body/document/type/sequence/filename/description/text/center[2]/div/table[3]/tbody/tr[2]/td[3]/p[3]

<document>
<type>SC 13D/A
<sequence>1
<filename>d39189dsc13da.htm
<description>SC 13D/A
<text>
<html><head>
<title>SC 13D/A</title>
</head>
<body bgcolor="WHITE">
<center><div align="left" style="width:8.5in">
<p style="line-height:1.0pt;margin-top:0pt;margin-bottom:0pt;border-bottom:1px solid #000000"> </p>
<p style="line-height:3.0pt;margin-top:0pt;margin-bottom:2pt;border-bottom:1px solid #000000"> </p> <p align="center" style="margin-top:4pt; margin-bottom:0pt; font-size:18pt; font-family:Times New Roman"><b>UNITED STATES </b></p>
<p align="center" style="margin-top:0pt; margin-bottom:0pt; font-size:18pt; font-family:Times New Roman"><b>SECURITIES AND EXCHANGE COMMISSION </b></p>
<p align="center" style="margin-top:0pt; margin-bottom:0pt; font-size:12pt; font-family:Times New Roman"><b>WASHINGTON, D.C. 20549 </b></p> <p style="font-size:18pt;margin-top:0pt;margin-bottom:0pt"> </p><center>
<p style="line-height:6.0pt;margin-top:0pt;margin-bottom:2pt;border-bottom:1.00pt solid #00000

In [275]:
soup_href = str(soup_hrefs)
form_link= 'https://www.sec.gov/Archives/'+re.findall(r'Archives(.+?)htm',soup_href)[0]+'htm'

df.loc[df['link'] == link,'Form_Link'] = form_link

'https://www.sec.gov/Archives//edgar/data/1091223/000119312520302746/d39189dsc13da.htm'

In [273]:
re.findall(r'Archives(.+?)htm',soup_href)[0]

'/edgar/data/1091223/000119312520302746/d39189dsc13da.'

In [63]:
import pandas as pd
from yahooquery import Ticker
import sqlite3
from bs4 import BeautifulSoup
from datetime import date
import matplotlib.pyplot as plt 
import textwrap
import numpy as np
import requests
import sys
from IPython.display import HTML
import contextlib
import pandas.io.formats.format as pf


@contextlib.contextmanager
def custom_formatting():
    orig_float_format = pd.options.display.float_format
    orig_int_format = pf.IntArrayFormatter

    pd.options.display.float_format = '{:0,.2f}'.format
    class IntArrayFormatter(pf.GenericArrayFormatter):
        def _format_strings(self):
            formatter = self.formatter or '{:,d}'.format
            fmt_values = [formatter(x) for x in self.values]
            return fmt_values
    pf.IntArrayFormatter = IntArrayFormatter
    yield
    pd.options.display.float_format = orig_float_format
    pf.IntArrayFormatter = orig_int_format
    
    


symbol = 'ODT'
symbol = symbol.upper()

import json

dil_columns = ['filingType',
 'date',
 'effective',           
 'sharesOffered',
 'aggregateAmount',
 'remaining',
 'mvphs',
 'float',
 'ib6',
 'price',
 'resaleAggregate',
 'resaleShares',
 'isHybrid',
 'warrants',
 'placementAgent',
 'subjectToCompletion',
 'isAmendment',
 'amendmentType',
 'AmountSold',
 'AtmIncreased',
 'addToRemaining',
 'markToOpen',
 'hasATM',
 'hasResale',
 'explanatoryNote',
 'hiddenATM',
 'foreign',
 'isIPO',
 'IsLatestPreliminary',
 'IsSupplement',
 'offeringInfo',
 'error',
 'fileID',
 'documentUrl',
 'fileUrl'                
          ]


url = "https://api.filingspro.com/sec?ticker="+symbol
resp = requests.get(url)
text = resp.text

d = json.loads(text)
if not d['shelfs']:
    print('No dilution data available')
else:
    # rows list initialization 
    rows = [] 

    # appending rows 
    for data in d["shelfs"]: 
        data_row = data['filings'] 
        time = data['effective'] 

        for row in data_row: 
            row['effective']= time 
            rows.append(row) 

    # using data frame 
    df = pd.DataFrame(rows)
    

    df.columns = pd.CategoricalIndex(df.columns, categories=dil_columns, ordered=True)
    df = df.sort_values(by=['date'], ascending = False)
    df = df.sort_index(axis=1)
    df = df.reset_index(drop=True)
    
    
    
    
#     df = df[df.columns.intersection(dil_columns)]
#     df = df[dil_columns]
    df['date'] = df['date'].astype('datetime64[ns]') 
    df['effective'] = df['effective'].astype('datetime64[ns]')
    df['effective'] = df['effective'].dt.date
    

    df = df.loc[:, ~(df.astype(str) == 'False').all()]
    df = df.loc[:, ~(df.astype(str) == '').all()]
#     df = df.sort_values(by=['date'],axis = 1,ascending = False, )

#     get a list of names
    effective_list =df['effective'].unique()
    fileID_list =df['fileID'].unique()
    for i in fileID_list:
        sub_df = df.loc[df.fileID == i]
        
        sub_df = sub_df.loc[:, ~(sub_df.astype(str) == 'False').all()]
        sub_df = sub_df.loc[:, ~(df.astype(str) == '').all()]
        try:
            sub_df = sub_df.sort_values(by=['date'], ascending = False)
        except Exception as e: 
            pass
        sub_df = sub_df.reset_index(drop=True)
        with custom_formatting():
            display(sub_df)

filingType       date effective  sharesOffered  aggregateAmount   remaining  \
0        S-3 2020-11-27       NaT              0      150,000,000 150,000,000   

   price  resaleAggregate warrants placementAgent  AmountSold  markToOpen  \
0  14.21                0     None                          0        True   

       fileID  \
0  333-250993   

                                                                     documentUrl  \
0  https://www.sec.gov/Archives/edgar/data/1717452/000156459020055322/odt-s3.htm   

                                                                                                fileUrl  
0  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-250993&owner=exclude&count=40

filingType       date   effective  sharesOffered  aggregateAmount  \
0      424B5 2020-08-31  2019-10-18      5,614,036       80,000,013   
1      424B5 2020-08-26  2019-10-18              0                0   
2        S-3 2019-09-27  2019-10-18              0      200,000,000   

    remaining  price  resaleAggregate warrants placementAgent  \
0 119,999,987  17.50                0       []      Jefferies   
1 200,000,000  17.50                0       []      Jefferies   
2 200,000,000  26.26                0     None                  

   subjectToCompletion  AmountSold  markToOpen      fileID  \
0                False           0        True  333-233990   
1                 True           0        True  333-233990   
2                False           0        True  333-233990   

                                                                        documentUrl  \
0  https://www.sec.gov/Archives/edgar/data/1717452/000156459020041861/odt-424b5.htm   
1  https://www.sec.gov/Archives/edgar/data/1717452/000156459020041370/odt-424b5.htm   
2     https://www.sec.gov/Archives/edgar/data/1717452/000156459019035768/odt-s3.htm   

                                                                                                fileUrl  
0  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-233990&owner=exclude&count=40  
1  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-233990&owner=exclude&count=40  
2  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-233990&owner=exclude&count=40

filingType       date   effective  sharesOffered  aggregateAmount  \
0      424B5 2019-06-27  2019-02-28      4,750,000      123,500,000   
1      424B5 2019-06-25  2019-02-28              0                0   
2        S-3 2019-02-22  2019-02-28              0      150,000,000   

    remaining  price  resaleAggregate warrants placementAgent  \
0  26,500,000  28.08                0       []      Jefferies   
1 150,000,000  28.08                0       []      Jefferies   
2 150,000,000  16.66                0     None                  

   subjectToCompletion  AmountSold  markToOpen      fileID  \
0                False           0        True  333-229785   
1                 True           0        True  333-229785   
2                False           0        True  333-229785   

                                                                        documentUrl  \
0  https://www.sec.gov/Archives/edgar/data/1717452/000156459019023862/odt-424b5.htm   
1  https://www.sec.gov/Archives/edgar/data/1717452/000156459019023475/odt-424b5.htm   
2     https://www.sec.gov/Archives/edgar/data/1717452/000156459019003809/odt-s3.htm   

                                                                                                fileUrl  
0  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-229785&owner=exclude&count=40  
1  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-229785&owner=exclude&count=40  
2  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-229785&owner=exclude&count=40

filingType       date   effective  sharesOffered  aggregateAmount  \
0      424B4 2017-12-08  2017-12-06      6,250,000      150,000,000   
1      S-1/A 2017-11-27  2017-12-06      5,880,000      182,574,000   
2        S-1 2017-11-13  2017-12-06              0      172,500,000   

   remaining  price  resaleAggregate warrants placementAgent  AmountSold  \
0          0   0.00                0       []      Jefferies           0   
1          0   0.00                0     None      Jefferies           0   
2          0   0.00                0     None      Jefferies           0   

   markToOpen  isIPO      fileID  \
0        True   True  333-221533   
1        True   True  333-221533   
2        True   True  333-221533   

                                                                            documentUrl  \
0  https://www.sec.gov/Archives/edgar/data/1717452/000119312517364946/d461398d424b4.htm   
1    https://www.sec.gov/Archives/edgar/data/1717452/000119312517351616/d461398ds1a.htm   
2     https://www.sec.gov/Archives/edgar/data/1717452/000119312517341109/d461398ds1.htm   

                                                                                                fileUrl  
0  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-221533&owner=exclude&count=40  
1  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-221533&owner=exclude&count=40  
2  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-221533&owner=exclude&count=40

filingType       date   effective  sharesOffered  aggregateAmount  \
0     S-1MEF 2017-12-06  2017-12-06         55,500       10,212,000   

   remaining  price  resaleAggregate warrants placementAgent  AmountSold  \
0 10,212,000   0.00                0     None                          0   

   markToOpen      fileID  \
0        True  333-221931   

                                                                            documentUrl  \
0  https://www.sec.gov/Archives/edgar/data/1717452/000119312517363030/d507118ds1mef.htm   

                                                                                                fileUrl  
0  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-221931&owner=exclude&count=40

In [45]:
df

filingType       date   effective  sharesOffered  aggregateAmount  \
3         S-3 2020-11-27         NaT              0        150000000   
0       424B5 2020-08-31  2019-10-18        5614036         80000013   
1       424B5 2020-08-26  2019-10-18              0                0   
2         S-3 2019-09-27  2019-10-18              0        200000000   
8       424B5 2019-06-27  2019-02-28        4750000        123500000   
9       424B5 2019-06-25  2019-02-28              0                0   
10        S-3 2019-02-22  2019-02-28              0        150000000   
4       424B4 2017-12-08  2017-12-06        6250000        150000000   
7      S-1MEF 2017-12-06  2017-12-06          55500         10212000   
5       S-1/A 2017-11-27  2017-12-06        5880000        182574000   
6         S-1 2017-11-13  2017-12-06              0        172500000   

    remaining  price  resaleAggregate warrants placementAgent  \
3   150000000  14.21                0     None                  
0   119999987  17.50                0       []      Jefferies   
1   200000000  17.50                0       []      Jefferies   
2   200000000  26.26                0     None                  
8    26500000  28.08                0       []      Jefferies   
9   150000000  28.08                0       []      Jefferies   
10  150000000  16.66                0     None                  
4           0   0.00                0       []      Jefferies   
7    10212000   0.00                0     None                  
5           0   0.00                0     None      Jefferies   
6           0   0.00                0     None      Jefferies   

    subjectToCompletion  AmountSold  markToOpen  isIPO      fileID  \
3                 False           0        True  False  333-250993   
0                 False           0        True  False  333-233990   
1                  True           0        True  False  333-233990   
2                 False           0        True  False  333-233990   
8                 False           0        True  False  333-229785   
9                  True           0        True  False  333-229785   
10                False           0        True  False  333-229785   
4                 False           0        True   True  333-221533   
7                 False           0        True  False  333-221931   
5                 False           0        True   True  333-221533   
6                 False           0        True   True  333-221533   

                                                                             documentUrl  \
3          https://www.sec.gov/Archives/edgar/data/1717452/000156459020055322/odt-s3.htm   
0       https://www.sec.gov/Archives/edgar/data/1717452/000156459020041861/odt-424b5.htm   
1       https://www.sec.gov/Archives/edgar/data/1717452/000156459020041370/odt-424b5.htm   
2          https://www.sec.gov/Archives/edgar/data/1717452/000156459019035768/odt-s3.htm   
8       https://www.sec.gov/Archives/edgar/data/1717452/000156459019023862/odt-424b5.htm   
9       https://www.sec.gov/Archives/edgar/data/1717452/000156459019023475/odt-424b5.htm   
10         https://www.sec.gov/Archives/edgar/data/1717452/000156459019003809/odt-s3.htm   
4   https://www.sec.gov/Archives/edgar/data/1717452/000119312517364946/d461398d424b4.htm   
7   https://www.sec.gov/Archives/edgar/data/1717452/000119312517363030/d507118ds1mef.htm   
5     https://www.sec.gov/Archives/edgar/data/1717452/000119312517351616/d461398ds1a.htm   
6      https://www.sec.gov/Archives/edgar/data/1717452/000119312517341109/d461398ds1.htm   

                                                                                                 fileUrl  
3   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-250993&owner=exclude&count=40  
0   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-233990&owner=exclude&count=40  
1   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-233990&owner=ex

In [7]:
sub_df

filingType       date  effective  sharesOffered  aggregateAmount  remaining  \
7     S-1MEF 2017-12-06 2017-12-06          55500         10212000   10212000   

   price  resaleAggregate warrants placementAgent  AmountSold  markToOpen  \
7   0.00                0     None                          0        True   

       fileID  \
7  333-221931   

                                                                            documentUrl  \
7  https://www.sec.gov/Archives/edgar/data/1717452/000119312517363030/d507118ds1mef.htm   

                                                                                                fileUrl  
7  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-221931&owner=exclude&count=40

In [48]:
effective_list

array([NaT, datetime.date(2019, 10, 18), datetime.date(2019, 2, 28),
       datetime.date(2017, 12, 6)], dtype=object)

In [36]:
df

filingType  \
0       424B5   
1       424B5   
2         S-3   
3         S-3   
4       424B4   
5       S-1/A   
6         S-1   
7      S-1MEF   
8       424B5   
9       424B5   
10        S-3   

                                                                             documentUrl  \
0       https://www.sec.gov/Archives/edgar/data/1717452/000156459020041861/odt-424b5.htm   
1       https://www.sec.gov/Archives/edgar/data/1717452/000156459020041370/odt-424b5.htm   
2          https://www.sec.gov/Archives/edgar/data/1717452/000156459019035768/odt-s3.htm   
3          https://www.sec.gov/Archives/edgar/data/1717452/000156459020055322/odt-s3.htm   
4   https://www.sec.gov/Archives/edgar/data/1717452/000119312517364946/d461398d424b4.htm   
5     https://www.sec.gov/Archives/edgar/data/1717452/000119312517351616/d461398ds1a.htm   
6      https://www.sec.gov/Archives/edgar/data/1717452/000119312517341109/d461398ds1.htm   
7   https://www.sec.gov/Archives/edgar/data/1717452/000119312517363030/d507118ds1mef.htm   
8       https://www.sec.gov/Archives/edgar/data/1717452/000156459019023862/odt-424b5.htm   
9       https://www.sec.gov/Archives/edgar/data/1717452/000156459019023475/odt-424b5.htm   
10         https://www.sec.gov/Archives/edgar/data/1717452/000156459019003809/odt-s3.htm   

        fileID  \
0   333-233990   
1   333-233990   
2   333-233990   
3   333-250993   
4   333-221533   
5   333-221533   
6   333-221533   
7   333-221931   
8   333-229785   
9   333-229785   
10  333-229785   

                                                                                                 fileUrl  \
0   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-233990&owner=exclude&count=40   
1   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-233990&owner=exclude&count=40   
2   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-233990&owner=exclude&count=40   
3   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-250993&owner=exclude&count=40   
4   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-221533&owner=exclude&count=40   
5   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-221533&owner=exclude&count=40   
6   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-221533&owner=exclude&count=40   
7   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-221931&owner=exclude&count=40   
8   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-229785&owner=exclude&count=40   
9   https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-229785&owner=exclude&count=40   
10  https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-229785&owner=exclude&count=40   

         date mvphs float ib6  aggregateAmount  sharesOffered  price  \
0  2020-08-31                         80000013        5614036  17.50   
1  2020-08-26                                0              0  17.50   
2  2019-09-27                        200000000              0  26.26   
3  2020-11-27                        150000000              0  14.21   
4  2017-12-08                        150000000        6250000   0.00   
5  2017-11-27                        182574000        5880000   0.00   
6  2017-11-13                        172500000              0   0.00   
7  2017-12-06                         10212000          55500   0.00   
8  2019-06-27                        123500000        4750000  28.08   
9  2019-06-25                                0              0  28.08   
10 2019-02-22                        150000000              0  16.66   

    resaleAggregate  remaining resaleShares  isHybrid warrants placementAgent  \
0                 0  119999987                  False       []      Jefferies   
1                 0  200000000                  False       []      Jefferies   
2                 0  200000000                  False     None         

In [42]:
def hf_change(symbol):   
    symbol = symbol.upper()
    base_url = "https://whalewisdom.com/stock/"+symbol #&type={}&dateb={}"
    edgar_resp = requests.get(base_url)#.format(cik))
    edgar_str = edgar_resp.text
    soup = BeautifulSoup(edgar_str, 'html.parser')
    table = soup.find_all('table')
    df_hf_change = pd.read_html(str(table))[1]
    return df_hf_change

In [43]:
hf_change('RGLS')

Q3 2020 All 13F Filers          Prior  Change Hedge Funds 1  \
0      In top 10:              1              0     NaN    1  (0.07%)   
1  Funds Holding:             28             35  -20.0%    8  (0.53%)   
2     13F shares:  7.815 Million  6.338 Million     NaN           NaN   

       Prior.1 Change.1  
0    0  (0.0%)      NaN  
1  11  (0.71%)  -27.27%  
2          NaN      NaN

In [44]:
table

[<table class="table table-condensed" style="font-size:11px;">
 <caption>Data as of 2021-01-09
   </caption>
 <tbody>
 <tr>
 <td><strong>Market Cap</strong></td><td>90.985 Million</td>
 <td><strong>Shares Outstanding</strong></td><td>66.901 Million</td>
 <td><strong>Avg 30-day Volume</strong></td><td>2.836 Million</td>
 </tr>
 <tr>
 <td><strong>P/E Ratio</strong></td><td>-1.9</td>
 <td><strong>Dividend Yield</strong></td><td></td>
 <td><strong>EPS</strong></td><td>-0.74</td>
 </tr>
 <tr>
 <td><strong>Price/Sales</strong></td><td>18.175</td>
 <td><strong>Price cash flow ratio</strong></td><td></td>
 <td><strong>Price free cash flow ratio</strong></td><td>-5.9</td>
 </tr>
 <tr>
 <td><strong>Book Value</strong></td><td>0.12</td>
 <td><strong>Price to Tangible Book</strong></td><td>11.91</td>
 <td><strong>Alpha</strong></td><td>-0.06</td>
 </tr>
 <tr>
 <td><strong>Short Interest Ratio</strong></td><td></td>
 <td><strong>% Short Interest to Float</strong></td><td></td>
 <td><strong>R-square